# 训练的可视化监控


## TensorboardX

Tensorboard 是 TensorFlow 中最常使用的可视化工具非，TensorboardX 使得 PyTorch 也享受到 Tensorboard 的便捷功能。

### 安装

安装 Tensorboard

```shell
pip install tensorboard
# or
conda install -c conda-forge --name mlab tensorboard
```

安装 TensorboardX

```shell
pip install tensorboardX
```

**PyTorch 1.8 之后的版本自带 TensorboardX，它被放在torch.utils.tensorboard中，因此无需多余配置。*


### 使用

为了使用 TensorboardX，我们首先需要创建一个 `SummaryWriter` 的实例，然后再使用`add_scalar`或`add_image`方法，将数字或图片记录到 SummaryWriter 实例中。

```
torch.utils.tensorboard.writer.SummaryWriter(log_dir=None)

add_scalar(tag, scalar_value, global_step=None, walltime=None)
```

log_dir 表示保存日志的路径，默认会保存在“runs/ 当前时间 _ 主机名”文件夹中。



In [1]:
from torch.utils.tensorboard import SummaryWriter
import numpy as np

# new ummaryWriter instance
writer = SummaryWriter()

for n_iter in range(100):
    # add_scalar ...
    writer.add_scalar('Loss/train', np.random.random(), n_iter)
    writer.add_scalar('Loss/test', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/train', np.random.random(), n_iter)
    writer.add_scalar('Accuracy/test', np.random.random(), n_iter)

img = np.zeros((3, 100, 100))
img[0] = np.arange(0, 10000).reshape(100, 100) / 10000
img[1] = 1 - np.arange(0, 10000).reshape(100, 100) / 10000

# add_image
writer.add_image('my_image', img, 0)
writer.close()

运行上面的代码后，会在当前目录下生成一个“runs”文件夹，里面存储了记录的数据。接着，只需要在当前目录下执行下面的命令，即可启动 Tensoboard。

```shell
tensorboard --logdir runs
#---
Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.12.0 at http://localhost:6006/ (Press CTRL+C to quit)
```

浏览器中打开 <http://localhost:6006/> ，

Tensorboard 中间区域就是上面 add_scalar 方法记录的 Loss 和 Accuracy。Tensorboard 已经按照迭代 step 绘制成了曲线图，可以非常直观地监控训练过程。

在“IMAGES”的标签页中，可以显示刚刚用 add_image 方法记录的图像数据。

#### 训练过程的可视化

模型定义

In [2]:
import random
import numpy as np
import torch
from torch import nn

# 模型定义
class LinearModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.weight = nn.Parameter(torch.randn(1))
    self.bias = nn.Parameter(torch.randn(1))

  def forward(self, input):
    return (input * self.weight) + self.bias

# 数据
w = 2
b = 3
xlim = [-10, 10]
x_train = np.random.randint(low=xlim[0], high=xlim[1], size=30)
y_train = [w * x + b + random.randint(0,2) for x in x_train]

训练

In [3]:
model = LinearModel()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4, weight_decay=1e-2, momentum=0.9)
y_train = torch.tensor(y_train, dtype=torch.float32)

writer = SummaryWriter()

for n_iter in range(500):
    input = torch.from_numpy(x_train)
    output = model(input)
    loss = nn.MSELoss()(output, y_train)
    model.zero_grad()
    loss.backward()
    optimizer.step()
    writer.add_scalar('Loss/train', loss, n_iter)

上面这段代码，记录了训练过程中的 Loss 的变换过程。可以看出 Loss 是一个下降的趋势，说明随着训练过程，模型越来越拟合我们的训练数据了。


## Visdom
